In [56]:
import h5py
import numpy as np
import torch 

file_dir = '/home/quang/Documents/XAI_env-main/data/processed/IMS/'
file_name = 'x_train.hdf5'

with h5py.File(file_dir + file_name, 'r') as file:
    dataset1 = file['x_train'][:]  # Replace 'dataset_name' with the actual dataset key or name
    dataset1_shape = dataset1.shape
    a = dataset1.dtype
print(dataset1_shape)
torch.is_tensor(dataset)
   

(7304, 20)


False

In [57]:
import h5py

file_dir = '/home/quang/Documents/XAI_env-main/data/processed/IMS/'
file_name = 'y_train.hdf5'

with h5py.File(file_dir + file_name, 'r') as file:
    dataset2 = file['y_train'][:]  # Replace 'dataset_name' with the actual dataset key or name
    dataset2_shape = dataset2.shape

print(dataset2_shape)

(7304, 3)


In [58]:
class CustomDataset:
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        current_sample = torch.tensor(self.data[idx, :], dtype=torch.float)
        current_target = torch.tensor(self.targets[idx], dtype=torch.long)
        return {
            "sample": current_sample,
            "target": current_target,
        }

In [59]:
data = dataset1
data.shape

(7304, 20)

In [60]:
targets = dataset2
targets.shape

(7304, 3)

In [61]:
custom_dataset = CustomDataset(data=data,targets=targets)

In [62]:
len(custom_dataset)

7304

In [63]:
print(custom_dataset)

In [64]:
custom_dataset[0]

{'sample': tensor([0.1569, 0.0183, 0.0194, 0.0166, 0.1277, 0.0036, 0.0034, 0.0016, 0.0035,
         0.0034, 0.0028, 0.0036, 0.0047, 0.0036, 0.0046, 0.0066, 0.0068, 0.0120,
         0.0117, 0.0073]),
 'target': tensor([34,  0, 10])}